In [1]:
import $cp.`/container/target/dependency/*`

import $cp.$

In [2]:

import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.kafka.clients.producer._

import org.testcontainers.containers.KafkaContainer
import org.testcontainers.utility.DockerImageName
import java.util.Properties

val spark = SparkSession.builder()
  .appName("ScalaSpark")
  .master("local[*]")
  .getOrCreate()

import spark.implicits._

val kafkaContainer = new KafkaContainer()

kafkaContainer.start()

val bootstrapServers = kafkaContainer.getBootstrapServers
val kafkaProps = new Properties()
kafkaProps.put("bootstrap.servers", bootstrapServers)
kafkaProps.put("key.serializer", "org.apache.kafka.common.serialization.StringSerializer")
kafkaProps.put("value.serializer", "org.apache.kafka.common.serialization.StringSerializer")

val kafkaProducer = new KafkaProducer[String, String](kafkaProps)

def sendToKafka(topic: String, values: Seq[String]): Unit = {
 
  values.foreach { value =>
    val record = new ProducerRecord[String, String](topic, value)
    kafkaProducer.send(record)
  }
  kafkaProducer.flush()
}

def readFromKafkaSpark(topic: String): DataFrame = {
  spark.read
  .format("kafka")
  .option("kafka.bootstrap.servers", bootstrapServers)
  .option("subscribe", topic)
  .load()
  .selectExpr("CAST(value AS STRING) as message")
}

val kafkaTopic = "my-topic-scala"


1 deprecation; re-run enabling -deprecation for details, or try -help
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


java.lang.IllegalStateException: Could not find a valid Docker environment. Please see logs and check configuration

In [ ]:
val data = Seq("hi", "there", "from", "scala")

sendToKafka(kafkaTopic, data)

In [ ]:
readFromKafkaSpark(kafkaTopic).show()